<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Semantic_text_splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipedia-api
!pip install nltk
!pip install sentence-transformers
!pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=07d05d968990b45c0b4d5b66da5ecbab8eff5dd4b2d943de9029bc512e9da339
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import wikipedia
import nltk
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import string
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')


# Initialize the Sentence Transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Define advanced stopwords and months/years
advanced_stopwords = set(stopwords.words('english')) | {'would', 'could', 'should', 'might', 'must', 'may', 'be', 'been', 'being'}
months_and_years = {'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december'}

# Function to fetch Wikipedia content
def fetch_wikipedia_document(query):
    try:
        content = wikipedia.page(query).content
        return content
    except wikipedia.exceptions.DisambiguationError as e:
        return "Disambiguation page found, please be more specific."

# Preprocessing function: Removing unwanted characters and stopwords
def preprocess_text(text):
    # Tokenize into sentences
    sentences = sent_tokenize(text)

    # Process each sentence
    processed_sentences = []
    for sentence in sentences:
        # Remove digits, special characters, punctuation, and months/years
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)  # Remove non-alphabetic characters
        words = word_tokenize(sentence.lower())  # Tokenize sentence into words and lower case
        words = [word for word in words if word not in advanced_stopwords and len(word) > 1 and word not in months_and_years]

        # Reassemble the sentence after filtering
        processed_sentences.append(' '.join(words))

    return processed_sentences

# Function to perform semantic text splitting based on cosine similarity
def semantic_text_splitter(text, threshold=0.75):
    processed_text = preprocess_text(text)  # Preprocess text
    # Generate embeddings for each sentence
    sentence_embeddings = model.encode(processed_text)

    # Initialize list to store the chunks
    sentence_splits = []
    current_chunk = processed_text[0]

    # Compare consecutive sentence embeddings
    for i in range(1, len(processed_text)):
        # Calculate cosine similarity between consecutive sentence embeddings
        similarity = cosine_similarity([sentence_embeddings[i-1]], [sentence_embeddings[i]])[0][0]

        # If similarity is lower than threshold, split the chunk
        if similarity < threshold:
            sentence_splits.append(current_chunk)
            current_chunk = processed_text[i]
        else:
            current_chunk += " " + processed_text[i]

    # Append the last chunk
    sentence_splits.append(current_chunk)

    return sentence_splits

# Main function to execute the workflow
def main():
    # Get topic input from the user
    user_input = input("Enter a topic to fetch from Wikipedia: ")

    # Fetch the document from Wikipedia
    document = fetch_wikipedia_document(user_input)
    if "Disambiguation page found" in document:
        print(document)
        return

    # Perform semantic text splitting
    chunks = semantic_text_splitter(document, threshold=0.75)

    # Show the chunks
    print("\nSemantically Split Text Chunks:\n")
    for idx, chunk in enumerate(chunks):
        print(f"Chunk {idx + 1}:\n{chunk}")
        print("\n" + "-"*50 + "\n")

# Run the main function
if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter a topic to fetch from Wikipedia: Artificial intelligence 

Semantically Split Text Chunks:

Chunk 1:
artificial intelligence ai broadest sense intelligence exhibited machines particularly computer systems field research computer science develops studies methods software enable machines perceive environment use learning intelligence take actions maximize chances achieving defined goals

--------------------------------------------------

Chunk 2:
machines called ais

--------------------------------------------------

Chunk 3:
highprofile applications ai include advanced web search engines eg google search recommendation systems used youtube amazon netflix interacting via human speech eg google assistant siri alexa autonomous vehicles eg waymo generative creative tools eg chatgpt ai art superhuman play analysis strategy games eg chess go

--------------------------------------------------

Chunk 4:
however many ai applications perceived ai lot cutting edge ai filtered general appl

In [ ]:
import wikipedia
import nltk
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Initialize the SentenceTransformer model for sentence embeddings
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Define advanced stopwords, months, and years
advanced_stopwords = set(stopwords.words('english')) | {'would', 'could', 'should', 'might', 'must', 'may', 'be', 'been', 'being'}
months_and_years = {'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december'}

# Function to fetch Wikipedia content
def fetch_wikipedia_document(query):
    try:
        content = wikipedia.page(query).content
        return content
    except wikipedia.exceptions.DisambiguationError as e:
        return "Disambiguation page found, please be more specific."

# Preprocessing function: removing unwanted characters and stopwords
def preprocess_text(text):
    sentences = sent_tokenize(text)  # Split text into sentences
    processed_sentences = []

    for sentence in sentences:
        # Remove digits, special characters, and punctuation
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
        words = word_tokenize(sentence.lower())  # Tokenize and lower case
        words = [word for word in words if word not in advanced_stopwords and len(word) > 1 and word not in months_and_years]
        processed_sentences.append(' '.join(words))  # Reassemble sentence
    return processed_sentences

# Function to perform semantic text splitting based on cosine similarity
def semantic_text_splitter(text, threshold=0.75, max_chunk_size=5):
    processed_text = preprocess_text(text)  # Preprocess the text into clean sentences
    sentence_embeddings = model.encode(processed_text)  # Generate embeddings for each sentence

    # Initialize variables to store chunks and the current chunk
    sentence_splits = []
    current_chunk = [processed_text[0]]  # Start with the first sentence in the first chunk

    # Iterate over the sentences to group semantically similar sentences
    for i in range(1, len(processed_text)):
        # Calculate cosine similarity between the last sentence of the current chunk and the next sentence
        similarity = cosine_similarity([sentence_embeddings[i-1]], [sentence_embeddings[i]])[0][0]

        # If similarity is high, we add the sentence to the current chunk
        if similarity >= threshold and len(current_chunk) < max_chunk_size:
            current_chunk.append(processed_text[i])
        else:
            # Otherwise, finalize the current chunk and start a new one
            sentence_splits.append(' '.join(current_chunk))
            current_chunk = [processed_text[i]]  # Start a new chunk with the current sentence

    # Add the last chunk if any remaining sentences exist
    if current_chunk:
        sentence_splits.append(' '.join(current_chunk))

    return sentence_splits

# Main function to run the workflow
def main():
    # Get input from the user for the topic to fetch from Wikipedia
    user_input = input("Enter a topic to fetch from Wikipedia: ")

    # Fetch the document content from Wikipedia
    document = fetch_wikipedia_document(user_input)
    if "Disambiguation page found" in document:
        print(document)
        return

    # Ask the user for a threshold and max chunk size
    threshold = float(input("Enter the semantic similarity threshold (0.0 to 1.0): "))
    max_chunk_size = int(input("Enter the maximum number of sentences per chunk: "))

    # Perform the semantic text splitting
    chunks = semantic_text_splitter(document, threshold, max_chunk_size)

    # Display the resulting chunks
    print("\nSemantically Split Text Chunks:\n")
    for idx, chunk in enumerate(chunks):
        print(f"Chunk {idx + 1}:")
        print(chunk)
        print("\n" + "-"*50 + "\n")

# Run the main function
if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter a topic to fetch from Wikipedia: Computer science 
Enter the semantic similarity threshold (0.0 to 1.0): 1.0
Enter the maximum number of sentences per chunk: 10

Semantically Split Text Chunks:

Chunk 1:
computer science study computation information automation

--------------------------------------------------

Chunk 2:
computer science spans theoretical disciplines algorithms theory computation information theory applied disciplines including design implementation hardware software

--------------------------------------------------

Chunk 3:
algorithms data structures central computer science

--------------------------------------------------

Chunk 4:
theory computation concerns abstract models computation general classes problems solved using

--------------------------------------------------

Chunk 5:
fields cryptography computer security involve studying means secure communication preventing security vulnerabilities

--------------------------------------------------

C

In [ ]:
import wikipediaapi
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import spacy

# Initialize the Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia( language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI,
    user_agent='My_Wikipedia_Bot/1.0 (my_email@example.com)')

nlp = spacy.load("en_core_web_sm")

# Initialize Hugging Face summarizer
summarizer = pipeline("summarization")

# Load stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to fetch document from Wikipedia
def fetch_wikipedia_article(query):
    page = wiki_wiki.page(query)
    if page.exists():
        return page.text
    else:
        return "Article not found."

# Function to clean and preprocess text
def preprocess_text(text):
    # Remove non-alphanumeric characters (punctuations, digits, months, years)
    text = re.sub(r'\b\d{1,4}\b', '', text)  # Removing years and digits
    text = re.sub(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\b', '', text)  # Remove month names
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters

    # Convert to lowercase and remove stopwords
    text = ' '.join([word.lower() for word in text.split() if word.lower() not in stop_words])

    return text

# Function to summarize text
def summarize_text(text):
    summary = summarizer(text)
    return summary[0]['summary_text']

# Function to split text into sentences
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

# Function to group sentences into chunks based on similarity
def group_sentences_into_chunks(sentences, threshold=0.7, chunk_size=3):
    vectorizer = TfidfVectorizer()
    chunks = []
    current_chunk = []

    # Compute similarity matrix
    for i in range(len(sentences)):
        sentence_vector = vectorizer.fit_transform([sentences[i]]).toarray()
        if current_chunk:
            similarity_matrix = cosine_similarity(sentence_vector, vectorizer.transform([' '.join(current_chunk)]).toarray())
            if similarity_matrix > threshold:  # Check similarity
                current_chunk.append(sentences[i])
            else:
                chunks.append(' '.join(current_chunk))  # Store the current chunk
                current_chunk = [sentences[i]]  # Start a new chunk
        else:
            current_chunk.append(sentences[i])

    if current_chunk:
        chunks.append(' '.join(current_chunk))  # Append the last chunk

    return chunks

# Function to extract keywords from text chunk
def extract_keywords_from_chunk(chunk, top_n=5):
    doc = nlp(chunk)
    keywords = [ent.text for ent in doc.ents]
    keywords = list(set(keywords))  # Remove duplicates

    return keywords[:top_n]

# Main function to integrate everything
def semantic_text_splitter(query, threshold=0.7, chunk_size=3):
    # Fetch Wikipedia article
    text = fetch_wikipedia_article(query)

    if text == "Article not found.":
        print(text)
        return

    # Preprocess text
    cleaned_text = preprocess_text(text)

    # Summarize text
    #summarized_text = summarize_text(cleaned_text)
    #print("Summarized Text:\n", summarized_text)

    # Split the summarized text into sentences
    sentences = split_into_sentences(cleaned_text)

    # Group sentences into chunks based on similarity threshold
    chunks = group_sentences_into_chunks(sentences, threshold, chunk_size)

    # Extract keywords from each chunk
    for idx, chunk in enumerate(chunks):
        keywords = extract_keywords_from_chunk(chunk)
        print(f"\nChunk {idx + 1}:\n", chunk)
        print("Keywords:", keywords)

# User input to define query and chunk size
query = input("Enter the topic to fetch from Wikipedia: ")
threshold = float(input("Enter threshold for sentence similarity (0.0 to 1.0): "))
chunk_size = int(input("Enter desired chunk size (number of sentences per chunk): "))

# Run the function
semantic_text_splitter(query, threshold, chunk_size)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter the topic to fetch from Wikipedia: Computer science 
Enter threshold for sentence similarity (0.0 to 1.0): 0.5
Enter desired chunk size (number of sentences per chunk): 10

Chunk 1:
 computer science study computation information automation computer science spans theoretical disciplines algorithms theory computation information theory applied disciplines including design implementation hardware software algorithms data structures central computer science theory computation concerns abstract models computation general classes problems solved using fields cryptography computer security involve studying means secure communication preventing security vulnerabilities computer graphics computational geometry address generation images programming language theory considers different ways describe computational processes database theory concerns management repositories data humancomputer interaction investigates interfaces humans computers interact software engineering focuses design prin

In [ ]:
!pip install yake==0.4.8

In [ ]:
import wikipediaapi
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import yake
import spacy
from yake import KeywordExtractor
# Initialize the Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(  language='en',
       extract_format=wikipediaapi.ExtractFormat.WIKI,
       user_agent='My_Wikipedia_Bot/1.0 (my_email@example.com)'
   )

# Initialize YAKE for keyword extraction
yake_extractor = yake.KeywordExtractor()

# Load stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to fetch document from Wikipedia
def fetch_wikipedia_article(query):
    page = wiki_wiki.page(query)
    if page.exists():
        return page.summary  # Fetches the summary for the article
    else:
        return "Article not found."

# Function to clean and preprocess text
def preprocess_text(text):
    # Remove non-alphanumeric characters (punctuations, digits, months, years)
    text = re.sub(r'\b\d{1,4}\b', '', text)  # Removing years and digits
    text = re.sub(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\b', '', text)  # Remove month names
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters

    # Convert to lowercase and remove stopwords
    text = ' '.join([word.lower() for word in text.split() if word.lower() not in stop_words])

    return text

# Function to split text into sentences
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

# Function to group sentences into chunks based on similarity
def group_sentences_into_chunks(sentences, threshold=0.7, chunk_size=3):
    vectorizer = TfidfVectorizer()
    chunks = []
    current_chunk = []

    # Compute similarity matrix
    for i in range(len(sentences)):
        sentence_vector = vectorizer.fit_transform([sentences[i]]).toarray()
        if current_chunk:
            similarity_matrix = cosine_similarity(sentence_vector, vectorizer.transform([' '.join(current_chunk)]).toarray())
            if similarity_matrix > threshold:  # Check similarity
                current_chunk.append(sentences[i])
            else:
                chunks.append(' '.join(current_chunk))  # Store the current chunk
                current_chunk = [sentences[i]]  # Start a new chunk
        else:
            current_chunk.append(sentences[i])

    if current_chunk:
        chunks.append(' '.join(current_chunk))  # Append the last chunk

    return chunks

# Function to extract keywords from text chunk using YAKE
def extract_keywords_from_chunk(chunk, top_n=5):
    keywords = yake_extractor.extract_keywords(chunk)
    keywords = [keyword[0] for keyword in keywords]  # Extract the keyword text

    return keywords[:top_n]

# Main function to integrate everything
def semantic_text_splitter(query, threshold=0.7, chunk_size=3):
    # Fetch Wikipedia article
    text = fetch_wikipedia_article(query)

    if text == "Article not found.":
        print(text)
        return

    # Preprocess text
    cleaned_text = preprocess_text(text)

    # Split the cleaned text into sentences
    sentences = split_into_sentences(cleaned_text)

    # Group sentences into chunks based on similarity threshold
    chunks = group_sentences_into_chunks(sentences, threshold, chunk_size)

    # Extract keywords from each chunk
    for idx, chunk in enumerate(chunks):
        keywords = extract_keywords_from_chunk(chunk)
        print(f"\nChunk {idx + 1}:\n", chunk)
        print("Keywords:", keywords)

# User input to define query and chunk size
query = input("Enter the topic to fetch from Wikipedia: ")
threshold = float(input("Enter threshold for sentence similarity (0.0 to 1.0): "))
chunk_size = int(input("Enter desired chunk size (number of sentences per chunk): "))

# Run the function
semantic_text_splitter(query, threshold, chunk_size)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter the topic to fetch from Wikipedia: Artificial intelligence 
Enter threshold for sentence similarity (0.0 to 1.0): 0.5
Enter desired chunk size (number of sentences per chunk): 10

Chunk 1:
 artificial intelligence ai broadest sense intelligence exhibited machines particularly computer systems field research computer science develops studies methods software enable machines perceive environment use learning intelligence take actions maximize chances achieving defined goals machines may called ais highprofile applications ai include advanced web search engines eg google search recommendation systems used youtube amazon netflix interacting via human speech eg google assistant siri alexa autonomous vehicles eg waymo generative creative tools eg chatgpt ai art superhuman play analysis strategy games eg chess go however many ai applications perceived ai lot cutting edge ai filtered general applications often without called ai something becomes useful enough common enough labeled ai any

Build semantic text splitter,  fetch document from wikipedia by user input  ,generate document summary by wikipedia summarizer,split into sentences, preprocess text exclude all advanced stopwords and special characters, digits,years,months,punctuatons, cluster sentences, create chunks and put all relevant similar clustered sentences into chunk,extract keywords from each chunk by using yake keyword extractor by taking number of keywords per chunk from user input,show output.

In [ ]:
!pip install transformers==4.31.0
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [ ]:
import wikipediaapi
import nltk
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re
import yake
from nltk.corpus import stopwords
import string
import numpy as np
import transformers

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

class SemanticTextSplitter:
    def __init__(self, model_name='all-MiniLM-L6-v2', similarity_threshold=0.7, num_keywords=5):
        self.model = SentenceTransformer(model_name)
        self.similarity_threshold = similarity_threshold
        self.num_keywords = num_keywords
        self.stop_words = set(stopwords.words('english'))

    def fetch_wikipedia_article(self, query):
        wiki = wikipediaapi.Wikipedia(language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI,
    user_agent='MySemanticSplitterBot/1.0 (my_email@example.com)'  # Replace with your info
)
        page = wiki.page(query)
        if page.exists():
            return page.text
        else:
            return None

    def preprocess_text(self, text):
        # Remove special characters, digits, years, months, and punctuation
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = re.sub(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December|year|month|day)\b', '', text, flags=re.IGNORECASE)
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = text.lower()

        # Remove stopwords
        words = text.split()
        words = [word for word in words if word not in self.stop_words]

        return ' '.join(words)

    def split_text_into_sentences(self, text):
        return nltk.sent_tokenize(text)

    def cluster_sentences(self, sentences):
        embeddings = self.model.encode(sentences)
        clusters = []
        current_cluster = [sentences[0]]

        for i in range(1, len(sentences)):
            similarity = cosine_similarity([embeddings[i-1]], [embeddings[i]])[0][0]
            if similarity < self.similarity_threshold:
                clusters.append(current_cluster)
                current_cluster = [sentences[i]]
            else:
                current_cluster.append(sentences[i])

        if current_cluster:
            clusters.append(current_cluster)

        return clusters

    def extract_keywords(self, chunk):
        # Use YAKE to extract keywords from the chunk
        extractor = yake.KeywordExtractor(lan='en', top=self.num_keywords)
        keywords = extractor.extract_keywords(chunk)
        return [keyword[0] for keyword in keywords]

    def generate_summary(self, text):
        # Use a pre-trained model for summarization (e.g., Hugging Face transformers)
        summarizer = transformers.pipeline("summarization", model="facebook/bart-large-cnn")
        summary = summarizer(text)
        return summary[0]['summary_text']

    def process_article(self, article_name):
        # Fetch and preprocess the Wikipedia article
        article_text = self.fetch_wikipedia_article(article_name)
        if not article_text:
            return "Article not found."

        # Summarize the article
       # summary = self.generate_summary(article_text)

        # Preprocess the summary
        cleaned_text = self.preprocess_text(article_text)

        # Split the text into sentences
        sentences = self.split_text_into_sentences(cleaned_text)

        # Cluster similar sentences
        clusters = self.cluster_sentences(sentences)

        # Extract keywords from each cluster and create chunks
        chunks = []
        for cluster in clusters:
            chunk_text = ' '.join(cluster)
            keywords = self.extract_keywords(chunk_text)
            chunks.append({
                'text': chunk_text,
                'keywords': keywords
            })

        return chunks


# Example usage
article_name = input("Enter the Wikipedia article name: ")
num_keywords = int(input("Enter the number of keywords per chunk: "))

splitter = SemanticTextSplitter(num_keywords=num_keywords)
result = splitter.process_article(article_name)

# Show results
if isinstance(result, str):  # Error message if the article was not found
    print(result)
else:
    for i, chunk in enumerate(result):
        print(f"\nChunk {i+1}:\n{chunk['text']}\nKeywords: {', '.join(chunk['keywords'])}\n")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter the Wikipedia article name: Computer science 
Enter the number of keywords per chunk: 5

Chunk 1:
computer science study computation information automation computer science spans theoretical disciplines algorithms theory computation information theory applied disciplines including design implementation hardware software algorithms data structures central computer science theory computation concerns abstract models computation general classes problems solved using fields cryptography computer security involve studying means secure communication preventing security vulnerabilities computer graphics computational geometry address generation images programming language theory considers different ways describe computational processes database theory concerns management repositories data humancomputer interaction investigates interfaces humans computers interact software engineering focuses design principles behind developing software areas operating systems networks embedded systems i

In [ ]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.7 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=1fe49801076b0ae4a57ce33e354baaeecf877f352c0d6d1b1dd426976e15fc9a
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=2544baf16d48682e417f83689d287c990f1cdb4eebd7b7c95637821ad952cf3d
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


:\\